In [1]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd

# ---------- 各種関数の定義 ---------- #
# 可到達行列を導出する関数
def reachability_matrix(matrix):

    ###----------可達行列作成処理----------###
    # 行列のサイズを取得
    n = len(matrix)

    # print('\n----------隣接行列----------')
    matrix = matrix.astype(bool)
    I = np.eye(n).astype(bool)
    adj_matrix = np.logical_or(matrix, I)
    # print(adj_matrix.astype(int))

    # print('\n----------可達行列----------')
    reachability_matrix = adj_matrix.copy()
    while not np.array_equal(reachability_matrix, reachability_matrix @ adj_matrix):
        reachability_matrix = reachability_matrix @ adj_matrix  #@演算子を利用することで行列積を求める（bool演算）
    # print(reachability_matrix.astype(int))

    return reachability_matrix

# ノードの配置を取得する関数
def get_layer_positions(removed_rows_dict, custom_pos={}):
    positions = {}
    current_x = 0
    for layer, nodes in removed_rows_dict.items():
        for node in nodes:
            positions[node] = (current_x, -layer)
            current_x += 1
        current_x = 0

    # custom_posで指定したノード番号のx座標を辞書リストに追加する
    for node, x_coord in custom_pos.items():
        positions[node] = (x_coord, positions[node][1])

    return positions

# グラフオブジェクトを作成する関数
def create_graph(removed_rows_dict, matrix):    # matrixを追加
    G = nx.DiGraph()

    # ノードをグラフに追加
    for i in range(matrix.shape[0]):
        G.add_node(i)

    # matrixを参照しエッジを追加する関数
    def add_edges(source_node, target_nodes, target_layer_dict):
        sorted_target_nodes = sorted(target_nodes, key=lambda node: target_layer_dict[node], reverse=True)
        for target_node in sorted_target_nodes:
            if source_node != target_node and not nx.has_path(G, source_node, target_node) and not nx.has_path(G, target_node, source_node):
                G.add_edge(source_node, target_node)
                print('Edge added:', source_node, '->' , target_node)

    sorted_removed_rows_dict = dict(sorted(removed_rows_dict.items()))  # 階層を昇順にソート

    # target_nodeが属する階層の情報を辞書リストとして取得
    target_layer_dict = {}
    for layer, nodes in removed_rows_dict.items():
        for node in nodes:
            target_layer_dict[node] = layer
    
    print('\n--------エッジ追加処理--------')
    for layer, nodes in sorted_removed_rows_dict.items():
        target_nodes = []
        for node in nodes:
            target_nodes = matrix[node].nonzero()[0]
            add_edges(node, target_nodes, target_layer_dict)
    
    return G

# R, A列を取得する関数
def get_RorA(reachability_matrix):
    R_or_A = []
    for i in range(len(reachability_matrix)):
        temp = []
        for j in range(len(reachability_matrix[i])):
            if reachability_matrix[i][j] == 1:
                temp.append(j)
        R_or_A.append(temp)
    return R_or_A

# 重複する要素を抽出する関数
def extract_duplicates(list1, list2):
    # 各リスト内の各サブリストの集合を作成
    set_a = [set(sublist) for sublist in list1]
    set_b = [set(sublist) for sublist in list2]

    # 各リスト内で同じ位置にあるサブリストから重複する要素を抽出
    duplicates = [list(set_a[i] & set_b[i]) for i in range(min(len(set_a), len(set_b)))]

    return duplicates

# 特定の数字を除去する関数
def remove_number(lst, num):
    return [x for x in lst if x != num]

# 新しいデータフレームを取得する関数
def get_new_df(df, i, removed_rows_dict):
    remove_idx = []
    remove_id = []
    remove_num = []
    for j in range(len(df)):
        if df['R'][j] == df['RandA'][j]:
            remove_idx.append(j)
            remove_id.append(df['id'][j])
            for num in df['R'][j]:
                remove_num.append(num)

    remove_idx, remove_num
    # print('remove_idx:', remove_idx)
    # print('remove_num:', remove_num)
    removed_rows_dict[i] = remove_id

    # dfを更新
    df = df.drop(remove_idx)

    # 特定の数字をリスト内からすべて除去する
    for num in remove_num:
        to_remove = num
        # print(num)
        # df = df.applymap(lambda x: [elem for elem in x if elem != to_remove])
        # df = df.applymap(lambda x: remove_number(x, to_remove))
        df['R'] = df['R'].apply(lambda x: remove_number(x, to_remove))
        df['A'] = df['A'].apply(lambda x: remove_number(x, to_remove))
        df['RandA'] = df['RandA'].apply(lambda x: remove_number(x, to_remove))

    return df.reset_index(drop=True), removed_rows_dict


# ここから本題

In [2]:
border = 6

A1path = f"C:\WorkSpace\collaborative_research\ISM_Analysis\ISMfiles\A1_b{border}.csv"
A2path = f"C:\WorkSpace\collaborative_research\ISM_Analysis\ISMfiles\A2_b{border}.csv"
Bpath =f"C:\WorkSpace\collaborative_research\ISM_Analysis\ISMfiles\B_b{border}.csv"
# Bpath = f""

A1matrix = np.genfromtxt(A1path, delimiter=',')
A2matrix = np.genfromtxt(A2path, delimiter=',')
Bmatrix = np.genfromtxt(Bpath, delimiter=',')

matrixes = [A1matrix, A2matrix, Bmatrix]

In [3]:
# ---------- 可達行列導出 ----------- #
reachability_matrixes = []

for matrix in matrixes:
    reach = reachability_matrix(matrix)
    reachability_matrixes.append(reach)

In [4]:
# ---------- データフレーム作成 ---------- #
df_list = []
for reachability_matrix in reachability_matrixes:
    R = []
    A = []
    id = list(range(len(reachability_matrix)))

    R = get_RorA(reachability_matrix)
    A = get_RorA(np.transpose(reachability_matrix))
    RandA = extract_duplicates(R, A)

    for RandA_i in RandA:
        RandA_i.sort()

    df = pd.DataFrame({'id':id, 'R':R, 'A':A, 'RandA':RandA})
    df_list.append(df)

In [5]:
# ---------- 関連度取得 ---------- #
d_add_r_list = []
for reachability_matrix in reachability_matrixes:
    # 各行の和を取得
    row_sums = np.sum(reachability_matrix, axis=1)    # ISM
    # row_sums = np.sum(matrix, axis=1)   # dematel

    # 各列の和を取得
    column_sums = np.sum(reachability_matrix, axis=0) # ISM
    # column_sums = np.sum(matrix, axis=0)    # dematel

    # 関連度
    d_add_r = row_sums + column_sums
    d_add_r_list.append(d_add_r)

In [6]:
# ---------- 階層を取得 ---------- #
levels_list = []
for df in df_list:
    levels = {}
    c = 0
    initial_df_length = len(df)

    for k in range(len(df)):
        df, levels = get_new_df(df, k+1, levels)
        # print('len(lebels): ', len(levels[i+1]))
        c += len(levels[k+1])
        # print('c: ', c)

        if c == initial_df_length:
            break
    levels_list.append(levels)

In [7]:
# ---------- ファイル名設定 ---------- #
def group(i):
    if i == 0:
        return "A1"
    elif i == 1:
        return "A2"
    else:
        return "B"

In [8]:
for i in range(3):
    # グラフを作成
    G = create_graph(levels_list[i], matrixes[i])

    # ノードの配置を取得
    pos = get_layer_positions(levels_list[i])

    # グラフを描画して表示
    plt.figure(figsize=(16, 16), dpi=600)
    nx.draw(G, pos, with_labels=True, node_size=1500, node_color="skyblue", font_size=10, font_weight="bold", arrowsize=15)
    plt.savefig('C:/WorkSpace/collaborative_research/ISM_Analysis/fig/'+group(i)+'/'+f'b{border}_'+'ISM-MODEL.png', format='png')
    plt.show()


--------エッジ追加処理--------
Edge added: 2 -> 22
Edge added: 12 -> 16
Edge added: 17 -> 8
Edge added: 18 -> 10
Edge added: 13 -> 18
Edge added: 28 -> 18
Edge added: 0 -> 13



--------エッジ追加処理--------
Edge added: 3 -> 16
Edge added: 6 -> 1



--------エッジ追加処理--------
Edge added: 1 -> 10
Edge added: 1 -> 60
Edge added: 13 -> 25
Edge added: 19 -> 16
Edge added: 27 -> 12
Edge added: 34 -> 86
Edge added: 46 -> 48
Edge added: 54 -> 10
Edge added: 64 -> 10
Edge added: 0 -> 13
Edge added: 0 -> 64
Edge added: 5 -> 1
Edge added: 5 -> 69
Edge added: 20 -> 19


In [9]:
# # ---------- ISMグラフ表示 ---------- #
# for i in range(len(levels_list)):
#     for j in range(len(levels_list[i])):
#         # グラフを作成
#         G = create_graph(levels_list[i][j], matrix[i][j])

#         # ノードの配置を取得
#         pos = get_layer_positions(levels_list[i][j])

#         # グラフを描画して表示
#         plt.figure(figsize=(16, 16), dpi=600)
#         nx.draw(G, pos, with_labels=True, node_size=1500, node_color="skyblue", font_size=10, font_weight="bold", arrowsize=15)
#         plt.savefig(name(i)+name2(j)+'ISM-MODEL.png', format='png')
#         plt.show()

#         # ---------- ノード位置の調整 ---------- #
#         # custom_posでノード番号に対応するx座標を指定する
#         # 自分で設定する場合
#         custom_pos = {3 : 1, 
#                     4 : 2,
#                     11: -1,
#                     16: 0}

#         # 関与度からcustom_posを取得する場合
#         custom_pos = {}
#         for k in range(len(matrix[i][j])):
#             custom_pos[k] = d_add_r_list[i][j][k]

#         # ノードの配置を取得
#         pos_fix = get_layer_positions(levels_list[i][j], custom_pos)

#         # グラフを描画して表示
#         plt.figure(figsize=(12, 10), dpi=600)
#         nx.draw(G, pos_fix, with_labels=True, node_size=1500, node_color="skyblue", font_size=10, font_weight="bold", arrowsize=15)
#         plt.savefig(name(i)+name2(j)+'ISM-MODEL_adjustment.png', format='png')
#         plt.show()


# for i in range(len(levels_list)):
#     # グラフを作成
#     G = create_graph(levels_list[i], matrix[i])

#     # ノードの配置を取得
#     pos = get_layer_positions(levels_list[i])

#     # グラフを描画して表示
#     plt.figure(figsize=(16, 16), dpi=600)
#     nx.draw(G, pos, with_labels=True, node_size=1500, node_color="skyblue", font_size=10, font_weight="bold", arrowsize=15)
#     plt.savefig(name(i)+name2(j)+'ISM-MODEL.png', format='png')
#     plt.show()

#     # ---------- ノード位置の調整 ---------- #
#     # 関与度からcustom_posを取得する場合
#     custom_pos = {}
#     for k in range(len(matrix[i][j])):
#         custom_pos[k] = d_add_r_list[i][j][k]

#     # ノードの配置を取得
#     pos_fix = get_layer_positions(levels_list[i][j], custom_pos)

#     # グラフを描画して表示
#     plt.figure(figsize=(12, 10), dpi=600)
#     nx.draw(G, pos_fix, with_labels=True, node_size=1500, node_color="skyblue", font_size=10, font_weight="bold", arrowsize=15)
#     plt.savefig(name(i)+name2(j)+'ISM-MODEL_adjustment.png', format='png')
#     plt.show()